<a href="https://colab.research.google.com/github/41371112h/114-1/blob/main/HW1_%E6%97%A5%E5%B8%B8%E6%94%AF%E5%87%BA%E9%80%9F%E7%AE%97%E8%88%87%E5%88%86%E6%94%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gspread
from gspread_dataframe import get_as_dataframe
from google.colab import auth
from google.auth import default
import pandas as pd
import re

# ====== 1. 授權 ======
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
print("已完成 Google 帳號授權")

# ====== 2. 開啟試算表 ======
sheet_url = "https://docs.google.com/spreadsheets/d/1HvCj-v296pFicLIjB5BbsX5U26Pd4d76QM5iuF1NeSo/edit?gid=0#gid=0"
sheet_id = re.search(r'/d/([a-zA-Z0-9-_]+)', sheet_url).group(1)

spreadsheet = gc.open_by_key(sheet_id)

worksheet = spreadsheet.worksheet("工作表1")
print("已打開工作表: Sheet1")

# ====== 3. 讀取現有資料到 DataFrame ======
df = get_as_dataframe(worksheet, evaluate_formulas=True, dtype=str)
df = df.dropna(how="all")
if df.empty:
    df = pd.DataFrame(columns=["日期", "時間", "品項", "金額"])

# ====== 4. 文字輸入並即時寫入 ======
records = []
while True:
    date = input("請輸入日期 (YYYY-MM-DD): ")
    time = input("請輸入時間 (HH:MM): ")
    item = input("請輸入品項: ")
    amount = input("請輸入金額: ")

    record = {"日期": date, "時間": time, "品項": item, "金額": float(amount)}
    records.append(record)
    df = pd.concat([df, pd.DataFrame([record])], ignore_index=True)

    worksheet.append_row([date, time, item, float(amount)])
    print("已新增一筆紀錄到 DataFrame 和 Google Sheet！\n")

# ====== 5. 列出所有紀錄 ======
print("\n--- 所有紀錄 ---")
for r in records:
    print(f"{r['日期']} {r['時間']} {r['品項']} ${r['金額']}")